In [1]:
from neural_networks import word_cnn, char_cnn, bd_lstm, lstm
import os
from read_files import split_imdb_files, split_yahoo_files, split_agnews_files
from word_level_process import word_process, get_tokenizer, text_to_vector_for_all
from config import config
from keras.preprocessing import sequence

dataset = "imdb"
model_name = "pretrained_word_cnn"

Using TensorFlow backend.


In [2]:
# Predict a string input with a model directly
# so skips the need of converting to sequence first...
def predict_str(model, s):
    maxlen = config.word_max_len[dataset]
    tokenizer = get_tokenizer(dataset)
    s_seq = tokenizer.texts_to_sequences([s])
    s_seq = sequence.pad_sequences(s_seq, maxlen=maxlen, padding='post', truncating='post')
    s_sep = s_seq[0]
    return model.predict(s_seq)

In [12]:
import stanfordnlp

# stanfordnlp.download('en')
nlp = stanfordnlp.Pipeline() # This sets up a default neural pipeline in English

Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': '/Users/weifanjiang/stanfordnlp_resources/en_ewt_models/en_ewt_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': '/Users/weifanjiang/stanfordnlp_resources/en_ewt_models/en_ewt_tagger.pt', 'pretrain_path': '/Users/weifanjiang/stanfordnlp_resources/en_ewt_models/en_ewt.pretrain.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': '/Users/weifanjiang/stanfordnlp_resources/en_ewt_models/en_ewt_lemmatizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
---
Loading: depparse
With settings: 
{'model_path': '/Users/weifanjiang/stanfordnlp_resources/en_ewt_models/en_ewt_parser.pt', 'pretrain_path': '/Users/weifanjiang/sta

In [3]:
model = word_cnn(dataset)
model_path = r'./runs/{}/{}.dat'.format(dataset, model_name)
model.load_weights(model_path)
print("successfully load model")

Build word_cnn model...




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where






successfully load model


In [4]:
# Data label:
# [1 0] is negative review
# [0 1] is positive review

train_texts, train_labels, test_texts, test_labels = split_imdb_files()
x_train, y_train, x_test, y_test = word_process(train_texts, train_labels, test_texts, test_labels, dataset)
print('successfully load data')

Processing IMDB dataset
successfully load data


In [11]:
import random
# idx = random.randint(0, x_test.shape[0] - 1)
idx = 15557

xi = x_test[idx:idx+1]
yi = y_test[idx:idx+1]
xi_text = test_texts[idx]

print(xi_text)
print()
print("model predict ", model.predict(xi)[0])
print("predict with predict_str ", predict_str(model, xi_text))
print("true label ", yi[0])

This is a stupid movie. When I saw it in a movie theater more than half the audience left before it was half over. I stayed to the bitter end. To show fortitude? I caught it again on television and it was much funnier. Still by no means a classic, or even consistently hilarious but the family kinda grew on me. I love Jessica Lundy anyway. If you've nothing better to do and it's free on t.v. you could do worse.

model predict  [0.92145205 0.07800014]
predict with predict_str  [[0.92145205 0.07800014]]
true label  [1 0]


In [14]:
doc = nlp(xi_text)

---